# WEATHER ANALYZER

In [2]:
import requests
import pandas as pd
import numpy as np

In [12]:
import json

In [1]:
# The function to call if we want to get input from user

"""def get_city():
    while True:
        city = input("Enter city name: ").strip()
        #Check if it's non-empty and only contains letters/spaces
        if city and city.replace(" ", "").isalpha():
            return city.title()  # Capitalize properly
        else:
            print("Invalid city name. Please enter a name using only letters and spaces.") """

'def get_city():\n    while True:\n        city = input("Enter city name: ").strip()\n        #Check if it\'s non-empty and only contains letters/spaces\n        if city and city.replace(" ", "").isalpha():\n            return city.title()  # Capitalize properly\n        else:\n            print("Invalid city name. Please enter a name using only letters and spaces.") '

In [3]:
city = "London" #We can call get_city to get city from the user after running above cell

In [4]:
API_KEY = "cf325cd195a148dd42688625d6e8809a"

In [69]:
url = f"https://api.openweathermap.org/data/2.5/forecast?q={city}&appid={API_KEY}&units=metric"

The API returns the 5-day forecast of the city given to the variable with all quantities in metric units

In [70]:
response = requests.get(url).json()

After checking the output json the whole forecast is retrieved by the list key

In [71]:
forecast = response["list"]

In [76]:
print(forecast)

[{'dt': 1751306400, 'main': {'temp': 27.68, 'feels_like': 27.27, 'temp_min': 27.68, 'temp_max': 27.68, 'pressure': 1014, 'sea_level': 1014, 'grnd_level': 1010, 'humidity': 38, 'temp_kf': 0}, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01d'}], 'clouds': {'all': 0}, 'wind': {'speed': 4.32, 'deg': 214, 'gust': 5.05}, 'visibility': 10000, 'pop': 0, 'sys': {'pod': 'd'}, 'dt_txt': '2025-06-30 18:00:00'}, {'dt': 1751317200, 'main': {'temp': 26.67, 'feels_like': 26.67, 'temp_min': 24.66, 'temp_max': 26.67, 'pressure': 1014, 'sea_level': 1014, 'grnd_level': 1010, 'humidity': 41, 'temp_kf': 2.01}, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01n'}], 'clouds': {'all': 0}, 'wind': {'speed': 0.95, 'deg': 192, 'gust': 1.17}, 'visibility': 10000, 'pop': 0, 'sys': {'pod': 'n'}, 'dt_txt': '2025-06-30 21:00:00'}, {'dt': 1751328000, 'main': {'temp': 24.33, 'feels_like': 24.1, 'temp_min': 22.66, 'temp_max': 24.33, 'pressure': 1015, 'sea_le

We normalize all the nested lists and disctionaries and explode them into respiective columns

In [77]:
df = pd.json_normalize(forecast, sep='_')

In [78]:
weather_df = pd.json_normalize(df['weather'].explode())
df = df.drop(columns=['weather']).reset_index(drop=True)
df = pd.concat([df, weather_df], axis=1)

In [79]:
df.head(5)

,dt,visibility,pop,dt_txt,main_temp,main_feels_like,main_temp_min,main_temp_max,main_pressure,main_sea_level,...,clouds_all,wind_speed,wind_deg,wind_gust,sys_pod,rain_3h,id,main,description,icon
0,1751306400,10000,0.0,2025-06-30 18:00:00,27.68,27.27,27.68,27.68,1014,1014,...,0,4.32,214,5.05,d,NaN,800,Clear,clear sky,01d
1,1751317200,10000,0.0,2025-06-30 21:00:00,26.67,26.67,24.66,26.67,1014,1014,...,0,0.95,192,1.17,n,NaN,800,Clear,clear sky,01n
2,1751328000,10000,0.0,2025-07-01 00:00:00,24.33,24.10,22.66,24.33,1015,1015,...,1,1.62,240,2.83,n,NaN,800,Clear,clear sky,01n
3,1751338800,10000,0.0,2025-07-01 03:00:00,20.39,20.21,20.39,20.39,1014,1014,...,8,1.75,308,3.85,n,NaN,800,Clear,clear sky,01n
4,1751349600,10000,0.0,2025-07-01 06:00:00,23.84,24.03,23.84,23.84,1015,1015,...,11,3.24,350,5.12,d,NaN,801,Clouds,few clouds,02d


In [80]:
df = df.drop(columns=['dt', 'clouds_all', 'visibility','pop','wind_deg','wind_gust','sys_pod','id','description','icon','rain_3h','main_temp_kf'])

In [81]:
df.head(5)

,dt_txt,main_temp,main_feels_like,main_temp_min,main_temp_max,main_pressure,main_sea_level,main_grnd_level,main_humidity,wind_speed,main
0,2025-06-30 18:00:00,27.68,27.27,27.68,27.68,1014,1014,1010,38,4.32,Clear
1,2025-06-30 21:00:00,26.67,26.67,24.66,26.67,1014,1014,1010,41,0.95,Clear
2,2025-07-01 00:00:00,24.33,24.10,22.66,24.33,1015,1015,1011,49,1.62,Clear
3,2025-07-01 03:00:00,20.39,20.21,20.39,20.39,1014,1014,1010,66,1.75,Clear
4,2025-07-01 06:00:00,23.84,24.03,23.84,23.84,1015,1015,1010,67,3.24,Clouds


In [82]:
df = df.rename(columns={
    'dt_txt': 'datetime',
    'main_temp': 'temp_C',
    'main_feels_like': 'feels_like_C',
    'main_temp_min': 'temp_min_C',
    'main_temp_max': 'temp_max_C',
    'main_pressure': 'pressure_hPa',
    'main_sea_level': 'sea_level_hPa',
    'main_grnd_level': 'ground_level_hPa',
    'main_humidity': 'humidity_percent',
    'main_temp_kf': 'temp_kf',
    'wind_speed': 'wind_speed_ms',
    'main': 'weather'
})

df.head(5)

,datetime,temp_C,feels_like_C,temp_min_C,temp_max_C,pressure_hPa,sea_level_hPa,ground_level_hPa,humidity_percent,wind_speed_ms,weather
0,2025-06-30 18:00:00,27.68,27.27,27.68,27.68,1014,1014,1010,38,4.32,Clear
1,2025-06-30 21:00:00,26.67,26.67,24.66,26.67,1014,1014,1010,41,0.95,Clear
2,2025-07-01 00:00:00,24.33,24.10,22.66,24.33,1015,1015,1011,49,1.62,Clear
3,2025-07-01 03:00:00,20.39,20.21,20.39,20.39,1014,1014,1010,66,1.75,Clear
4,2025-07-01 06:00:00,23.84,24.03,23.84,23.84,1015,1015,1010,67,3.24,Clouds


In [83]:
df['datetime'] = pd.to_datetime(df['datetime'])

After dropping all unimportant columns, renaming columns and ensuring consistent data types the data is ready for numpy analysis

In [84]:
df.head(5)

,datetime,temp_C,feels_like_C,temp_min_C,temp_max_C,pressure_hPa,sea_level_hPa,ground_level_hPa,humidity_percent,wind_speed_ms,weather
0,2025-06-30 18:00:00,27.68,27.27,27.68,27.68,1014,1014,1010,38,4.32,Clear
1,2025-06-30 21:00:00,26.67,26.67,24.66,26.67,1014,1014,1010,41,0.95,Clear
2,2025-07-01 00:00:00,24.33,24.10,22.66,24.33,1015,1015,1011,49,1.62,Clear
3,2025-07-01 03:00:00,20.39,20.21,20.39,20.39,1014,1014,1010,66,1.75,Clear
4,2025-07-01 06:00:00,23.84,24.03,23.84,23.84,1015,1015,1010,67,3.24,Clouds


Columns are converted into numpy arrays and then basic numpy functions are used.

In [85]:
temps = df['temp_C'].to_numpy()
winds = df['wind_speed_ms'].to_numpy()
humidity = df['humidity_percent'].to_numpy()

In [86]:
mean_temp = np.mean(temps)
max_temp = np.max(temps)
min_temp = np.min(temps)
std_temp = np.std(temps)

mean_wind = np.mean(winds)
max_wind = np.max(winds)

mean_humidity = np.mean(humidity)

In [87]:
print(f"Mean Temp: {mean_temp:.2f} °C")
print(f"Max Temp: {max_temp:.2f} °C")
print(f"Min Temp: {min_temp:.2f} °C")
print(f"Temp Std Dev: {std_temp:.2f} °C")
print(f"Mean Wind Speed: {mean_wind:.2f} m/s")
print(f"Max Wind Speed: {max_wind:.2f} m/s")
print(f"Mean Humidity: {mean_humidity:.2f} %")

Mean Temp: 22.05 °C
Max Temp: 33.26 °C
Min Temp: 14.70 °C
Temp Std Dev: 4.63 °C
Mean Wind Speed: 2.98 m/s
Max Wind Speed: 5.65 m/s
Mean Humidity: 49.88 %
